In [6]:
import pandas as pd
import re
from concurrent.futures import ThreadPoolExecutor

import numpy as np
import pandas as pd
from astropy.coordinates import SkyCoord
from astropy.time import Time
from astroquery.jplhorizons import Horizons


# df = pd.read_csv('D:\Project\Earthquake\Comets\\raw_asteroid\AsteroidCT.csv')
df = pd.read_csv('D:\Project\Earthquake\Comets\\raw_asteroid\CLEAR101_P.csv')            # round 2
df['Time'] = pd.to_datetime(df['Time'])

In [7]:
def calculate_position(obj, time):
    """
    Args: 
        obj: obj name str
        time: time (utc) format yyyy-mm-dd
    """
    try:
        epoch = Time(time)
        q = Horizons(obj, location='@0', epochs=epoch.tdb.jd)
        tab = q.vectors(refplane='earth')
        c = SkyCoord(tab['x'].quantity, tab['y'].quantity, tab['z'].quantity,
                     representation_type='cartesian', frame='icrs',
                     obstime=epoch)
        result = str(c)
        matches = re.findall(r'-?\d+\.\d+', result)
        result_list = [float(match) for match in matches]
        return result_list
    except Exception as e:
        print(f"Error calculating position for object {obj} at time {time}")
        return [99,99,99]

no_use = ['-74', '-49','-49', '-490', '-255' , '2000016', '20065803', '-64']

def process_chunk(chunk):
    chunk[['X_new_1', 'Y_new_1', 'Z_new_1']] = chunk.apply(
        lambda row: pd.Series(calculate_position(row['Object_Name'], row['Time'])), axis=1)
    return chunk


no_use = ['-74', '-49','-49', '-490', '-255' , '2000016', '20065803', '-64']

In [8]:

df = df[(df['X'] == 99) & (df['Y'] == 99) & (df['Z'] == 99) & (~df['Object_Name'].isin(no_use))]
df

,Time,Object_Name,X_new,Y_new,Z_new,X,Y,Z
0,2001-12-28,Braille,99,99,99,99.0,99.0,99.0
2,2001-12-31,699,99,99,99,99.0,99.0,99.0
6,2002-01-02,399,99,99,99,99.0,99.0,99.0
7,2002-01-03,399,99,99,99,99.0,99.0,99.0
8,2002-01-03,Polymele,99,99,99,99.0,99.0,99.0
...,...,...,...,...,...,...,...,...
24845,2023-12-22,Leucus,99,99,99,99.0,99.0,99.0
24846,2023-12-22,Polymele,99,99,99,99.0,99.0,99.0
24847,2023-12-22,Lutetia,99,99,99,99.0,99.0,99.0
24848,2023-12-29,-122911,99,99,99,99.0,99.0,99.0


In [9]:
num_threads = 8
chunks = np.array_split(df, num_threads)

# Process each part concurrently using 8 threads
with ThreadPoolExecutor(max_workers=num_threads) as executor:
    results = list(executor.map(process_chunk, chunks))


df = pd.concat(results)

df.to_csv('CLEAR101_P1.csv', index=False)

Error calculating position for object Braille at time 2001-12-28 00:00:00
Error calculating position for object Gaspra at time 2020-10-08 00:00:00
Error calculating position for object Gaspra at time 2019-09-20 00:00:00
Error calculating position for object Eros at time 2011-01-21 00:00:00
Error calculating position for object Bennu at time 2014-05-13 00:00:00
Error calculating position for object Vesta at time 2020-10-09 00:00:00
Error calculating position for object 599 at time 2019-09-21 00:00:00
Error calculating position for object Bennu at time 2007-06-02 00:00:00
Error calculating position for object Braille at time 2017-01-01 00:00:00
Error calculating position for object 699 at time 2019-09-21 00:00:00
Error calculating position for object 301 at time 2007-06-03 00:00:00
Error calculating position for object Annefrank at time 2017-01-01 00:00:00
Error calculating position for object Eurybates at time 2011-01-22 00:00:00
Error calculating position for object Ryugu at time 2007-